# Assess Mental State Changes with Remote Photoplethysmography and Thermal Imaging

This notebook replicates all steps in the article.

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
source_folder_path = '../resource/video/video_source/' 
target_folder_path = '../result/remote_ppg/'

In [2]:
# 需要決定的參數
step = 21
method = "rf"
mode = "rPPG"
input_file_path = os.path.join(target_folder_path, "bpm_output","summary","result_updated.csv")
output_file_path = os.path.join(target_folder_path, "bpm_output","summary", "{}_{}_{}_results.xlsx".format(mode, method, step))
save_path_bar = os.path.join(target_folder_path, "bpm_output","summary", "SHAP_plot_{}_{}_{}_results.png".format(mode, method, step))
save_path_dot = os.path.join(target_folder_path, "bpm_output","summary", "SHAP_{}_{}_{}_results.png".format(mode, method, step))
hrv_variable_list = ["HR", "SDNN", "rMSSD", "pNN50", "ln(HF)", "ln(LF)", "ln(LF/HF)"]
hrv_difference_list = [i + "_difference" for i in hrv_variable_list]
features = hrv_variable_list
add_principal_component_analysis = False
n_principal_component_list = [int(x) for x in np.linspace(3, 15, num=13)]
parameter_grid_search = False
select_important_feature = False
n_parameter_grid_search_iteration = 100
#repeat_times = 1
parameter_search_fold = 5
#n_fold = 10

In [3]:
# n_principal_component_list = [int(x) for x in np.linspace(3, 15, num=13)]
def model_preparation():
    n_parameter_grid_search_iteration = 100
    parameter_search_fold = 5
    my_random_state = 42
    n_estimators = [int(x) for x in np.linspace(start=100, stop=1000, num=3)]  # default=100
    max_depth = [int(x) for x in np.linspace(2, 12, num=6)]
    max_depth.append(None)
    min_samples_split = [int(x) for x in np.linspace(2, 22, num=6)]
    min_samples_leaf = [int(x) for x in np.linspace(2, 12, num=6)]
    bootstrap = [True, False]
    svc_grid = {'C': [x for x in np.linspace(0.1, 10, num=100)],
                'gamma': [x for x in np.linspace(0.001, 0.1, num=100)],
                'kernel': ['rbf', 'poly', 'sigmoid']}

    random_grid = {'n_estimators': n_estimators,  # 'max_features': max_features,
                'max_depth': max_depth, 'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}
    forest2 = RandomForestClassifier(random_state=my_random_state)
    svc_model = svm.SVC()
    rf_random = RandomizedSearchCV(estimator=forest2, param_distributions=random_grid,
                                n_iter=n_parameter_grid_search_iteration, cv=parameter_search_fold, verbose=2,
                                random_state=my_random_state)
    svc_random = RandomizedSearchCV(estimator=svc_model, param_distributions=svc_grid,
                                    n_iter=n_parameter_grid_search_iteration, cv=parameter_search_fold, verbose=2,
                                    random_state=my_random_state)
    return rf_random, svc_random

In [4]:
def read_dataset(columns, path):
    df = pd.read_csv(path)
    df_data = df.loc[:, columns]
    df_data.dropna(axis=0, how='all', inplace=True)  # 删掉全是空值的行
    x = df_data.fillna(df_data.median())  # 插值法
    y = df.loc[x.index, ['Step']]
    remeber_no = df.loc[x.index, ["No-Step"]]
    return x, y, remeber_no

In [5]:
x_read, y_read, remeber_no = read_dataset(features, input_file_path)
rf_random, svc_random = model_preparation()
scaler = StandardScaler()
# transform data
x_read_scaled = scaler.fit_transform(x_read)
x_read_scaled = pd.DataFrame(data=x_read_scaled)

In [6]:
import sys
sys.path.insert(0, sys.path[0]+"/../")

from researchtoolbox.ml import cross_validation as cv
from researchtoolbox.ml import supervised_learning as svl

cross_validation_settings = cv.cross_validation_setting_model()
cross_validation_settings.n_fold = 10
cross_validation_settings.principal_component_list = []
cross_validation_settings.repeat_times = 1

data_model_settings = svl.supervised_learning_data_model()
data_model_settings.x = x_read_scaled
data_model_settings.Y  = y_read
data_model_settings.remeber_no = remeber_no

cv.cross_validation().do_cross_validation(cross_validation_settings,data_model_settings, rf_random)

d:\Anaconda3\envs\researchtoolbox\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
d:\Anaconda3\envs\researchtoolbox\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks,

  0%|          | 0/103 [00:00<?, ?it/s]

divide by zero encountered in log
invalid value encountered in multiply
divide by zero encountered in log
invalid value encountered in multiply
divide by zero encountered in log
invalid value encountered in multiply
divide by zero encountered in log
invalid value encountered in multiply
divide by zero encountered in log
invalid value encountered in multiply
divide by zero encountered in log
invalid value encountered in multiply
divide by zero encountered in log
invalid value encountered in multiply
divide by zero encountered in log
invalid value encountered in multiply


0.2621359223300971


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].